###  

In [ ]:
#OpenAI API Credentials einsetzen
org_key = ""
api_key = ""

# ArcGIS Notebooks & ChatGPT - Automatisierte Stadtführung


### Top 10 Sehenswürdigkeiten von ChatGPT erfragen

In [ ]:
city = "Bonn"
country = "Deutschland"
rap = True

### Bibliotheken installieren und mit GIS verbinden:

In [ ]:
import pandas as pd
import numpy as np
import re
import base64
import os
import shutil
from arcgis.features import GeoAccessor
from arcgis.gis import GIS
gis = GIS("home")

###  OpenAI Python Bibliothek installieren:

In [ ]:
#In ArcGIS Online sollten keine weiteren Bibliotheken notwenidig sein
!pip install openai

###  Import OpenAI Bibliothek, Verbindung zur API herstellen & Unterhaltung mit ChatGPT beginnen :

In [ ]:
from openai import OpenAI
client = OpenAI(organization=org_key,api_key=api_key)

In [ ]:
# Liste der 10 besten Sehenswürdigkeiten erfragen und Response verarbeiten
sights_raw_response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content":"Schreib mir eine Liste, ohne Beschreibungen, der 10 besten Sehenswürdigkeiten in "+ city +", "+country}])
sights_response = sights_raw_response.model_dump()
sights = sights_response["choices"][0]["message"]["content"]

In [ ]:
# DataFrame aus Response erstellen und Daten aufbereiten
sights_df = pd.Series(sights.split('\n'),name="stops").to_frame()
sights_df["stops"].replace('', np.nan, inplace=True)
sights_df.dropna(subset=['stops'], inplace=True)
sights_df

### Beschreibung zu den Sehenswürdigkeiten hinzufügen

In [ ]:
#Für jede Sehenswürdigkeit eine Beschreibung abfragen und verarbeiten
sights_df['description'] = sights_df.apply(lambda _: '', axis=1)
for index, row in sights_df.iterrows():
    # Aufzählung von Liste, sowie weitere Informationen in Klammern / mit - oder : ersetzen
    row["stops"] = re.sub('[0-9]+.|\(.*|\:.*|\-.*','',row["stops"])
    # Stadt und Land hinzufügen um bessere Beschreibung zu erhalten
    sights_df.loc[index,"stops"] = row["stops"] + ', '+city+ ', '+country
    # Beschreibung abfragen und verarbeiten
    description_raw_response = client.chat.completions.create(model="gpt-3.5-turbo-1106", messages=[{"role": "user", "content":"Bitte gib mir eine kurze Beschreibung zu " +row["stops"]}])
    description_response = description_raw_response.model_dump()
    description = description_response["choices"][0]["message"]["content"].replace("\n", "")
    sights_df.loc[index, "description"] = description
    print("Quassel mit ChatGPT über "+row["stops"])
sights_df

### Geokodieren der Sehenswürdigkeiten in einen Spatially Enabled Data Frame:

In [ ]:
# Geokodierung der Sehenswürdigkeiten auf Basis des des  Namens + Stadt + Land
sights_sdf = pd.DataFrame.spatial.from_df(sights_df, address_column='stops')
sights_sdf = sights_sdf[["stops","description","Score","Match_addr","LongLabel","Addr_type","City","Country","X","Y","SHAPE"]]
# Stadt und Land wieder von Sehenswürdigkeitsnamen entfernen
a = ', '+city+', '+country
sights_sdf['stops'] = sights_sdf['stops'].apply(lambda x: x.replace(a,""))
sights_sdf

### Feature Service publizieren:

In [ ]:
# Feature Service in Ordner OpenAI im aktiven GIS publizieren
name = "Stadtführung in "+city
sights_fs = sights_sdf.spatial.to_featurelayer(name,sanitize_columns=True,folder="OpenAI")

### Bilder von OpenAI (DALL.E) malen lassen:

In [ ]:
# Falls Ordner für Stadt bereits existiert, diesen löschen und ersetzen (könnte stattdessen natürlich auch wieder genutzt werden)
if os.path.exists('/arcgis/home/'+city):
    shutil.rmtree('/arcgis/home/'+city)
os.mkdir('/arcgis/home/'+city)
# Für jeden Stop ein Bild über die Beschreibung der Sehenswürdigkeit von DALL.E malen lassen
for index, row in sights_df.iterrows():
    row["stops"] = re.sub('^\s|\s$','',(re.sub('\(.*','',row["stops"]))).replace(a,"")
    print("DALL.E malt die/den/das "+row["stops"])
    # Falls das Bild aufgrund der Content-Policy Violation nicht gemalt werden darf, male eine lustige Katze.
    try:
        img_rsp = client.images.generate(prompt=row["description"],n=1,size="256x256",response_format="b64_json")
    except Exception as e:
        print("Fehler: "+row["stops"]+" konnte nicht gemalt werden und wird durch eine lustige Katze ersetzt.", e)
        img_rsp = client.images.generate(prompt="Lustige Katze",n=1,size="256x256",response_format="b64_json")
    # Response von base64 String zu jpg Datei umwandeln und lokal speichern
    img_data = base64.b64decode(img_rsp.model_dump()["data"][0]["b64_json"])
    img_file = '/arcgis/home/'+city+'/'+row["stops"]+'.jpg'
    with open(img_file, 'wb') as f:
        f.write(img_data)

### Bilder an publizierten Feature Service anhängen:

In [ ]:
# Attachements aktivieren
sights_fs.layers[0].manager.update_definition({"hasAttachments": True})
# FID und Name der Sehenswürdigkeiten abfragen
features = sights_fs.layers[0].query(where='1=1',out_fields=["FID","stops"],return_geometry=False).features

In [ ]:
# Name des Features / Sehenswürdigkeit aus dem publizierten FS mit dem lokal gespeicherten Bild matchen und dieses anhängen
for feature in features:
    try:
        sights_fs.layers[0].attachments.add(feature.attributes["FID"],'/arcgis/home/'+city+'/'+re.sub('^\s|\s$','',(re.sub('\(.*','',feature.attributes["stops"].replace(a,""))))+'.jpg')
    except:
        print("Konnte Bild nicht dem Feature zuordnen")

### ArcGIS StoryMap auf Basis des publizierten Feature Service erstellen:

In [ ]:
from arcgis.apps.storymap import StoryMap
# StoryMap Template abfragen
sm_template = StoryMap("578a1e1e8d6e4657a25ea75af1f7e06a")
sm_template

In [ ]:
# StoryMap dublizieren
sm_new_item = sm_template.duplicate(name)
sm_new = StoryMap(sm_new_item.id)

In [ ]:
# In neuer StoryMap Name, Beschreibung und FeatureService austauschen (To Do: In neuem Python API Release über Funktionen statt nodes möglich)
sm_new.properties["nodes"]['n-pqjjwO']["data"]["title"] = name
sm_new.properties["nodes"]['n-pqjjwO']["data"]["summary"] = name +" erstellt mit ArcGIS Notebooks & OpenAI"
sm_new.properties["resources"]["r-hUErSr"]["data"]["itemId"] = sights_fs.id

In [ ]:
# Falls Rap auf True steht noch einen Rap auf Englisch über die Sehenswürdigkeiten abfragen, da ChatGPT auf Deutsch nicht gut reimen kann. Diesen ebenfalls in StoryMap einpflegen.
if rap:
    rap_raw_response = client.chat.completions.create(model="gpt-3.5-turbo-1106", messages=[{"role": "user", "content":"Switch to Englisch and please rap about the Top 10 sights in " +city}])
    rap_response = rap_raw_response.model_dump()
    rap_lyrics = rap_response["choices"][0]["message"]["content"]
    sm_new.properties["nodes"]['n-pGga26']["data"]['text'] = '<strong>Easter Egg:</strong><br><i>"Hey Chat-GPT, please rap about the Top 10 sights in '+city+'"</i>!<br><br>'+rap_lyrics
else:
    sm_new.get('n-pGga26').delete()

In [ ]:
# Neue StoryMap speichern und publizieren
sm_new.save(name, publish=True)